Comenzamos por importar las librerias pertinentes yy definir la ruta de estos

In [ ]:
import re
import pandas as pd
import numpy  as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ruta_items = '/content/drive/MyDrive/TP N1 Henry/EDA/steam_items.csv'
ruta_juegos = '/content/drive/MyDrive/TP N1 Henry/EDA/steam_games.csv'
ruta_reviews = '/content/drive/MyDrive/TP N1 Henry/EDA/steam_reviews.csv'


pd_items = pd.read_csv(ruta_items)
pd_reviews = pd.read_csv(ruta_reviews)
pd_games = pd.read_csv(ruta_juegos)

<ipython-input-34-6ae08b9fba24>:7: DtypeWarning: Columns (1,2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_reviews = pd.read_csv(ruta_reviews)


# A continuacion se hará otro proceso de refinado de datos #

comenzaremos por desarmar las listas te *pd_games* presentes en la tabla specs y tags, luego pondremos los valores de id
como entero

In [ ]:
pd_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,643980.0,Secret Level SRL


Añadimos a *pd_items* las columnaas de tags y specs del dataframe *pd_games*

In [ ]:
merged_df = pd.merge(pd_items, pd_games[['id', 'tags', 'specs']], left_on='item_id', right_on='id', how='left')
merged_df = merged_df.drop(columns='id')
pd_items = merged_df

en *pd_reviews* quitamos los valores nulos

en la columna ¨posted¨ la cual tiene un formato :posted (fecha), se quita ese posted y luego se transforma a fecha

algunas fechas no disponen de año, por lo cual se les asigna el año 2016 ya que el registro mas nuevo es de dic 2015

se cambia el formato de id para que contenga valores enteros

se quita la columna helpful puesto que su valor importante esta en rev_utility

In [ ]:
pd_reviews = pd_reviews.dropna() \
                       .assign(posted=lambda x: x['posted'].str.replace('Posted', '') \
                                                     .apply(lambda x: x + ', 2016' if isinstance(x, str) and ', ' not in x else x)) \
                       .assign(posted=lambda x: pd.to_datetime(x['posted'], errors='coerce')) \
                       .assign(item_id=lambda x: x['item_id'].astype(int)) \
                       .sort_values(by='posted', ascending=False) \
                       .reset_index(drop=True) \
                       .drop(columns='helpful')


comenzaremos con la creacion de tablas

##def PlayTimeGenre( genero : str ):
**Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}**


crearemos una tabla que contenga el id del juego, los tags, el año de lanzamiento y la suma de horas jugadas por todos los usuarios ese juego

In [ ]:
dat_func1 = pd_items.groupby('item_id').agg({
    'playtime_forever': 'sum',
    'playtime_2weeks': 'sum'
}).reset_index()
dat_func1 = pd.merge(dat_func1, pd_games[['id', 'release_date','genres']], left_on='item_id', right_on='id', how='inner')
dat_func1 = dat_func1.drop(columns=['id'])
dat_func1['genres'] = dat_func1['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else 'desconocido' if pd.isnull(x) else x)
dat_func1['genres'] = dat_func1['genres'].str.strip("[]").astype(str)
dat_func1

,item_id,playtime_forever,playtime_2weeks,release_date,genres
0,10,17386015,48774,2000-11-01,'Action'
1,20,961702,14117,1999-04-01,'Action'
2,30,758991,14042,2003-05-01,'Action'
3,40,154486,38,2001-06-01,'Action'
4,50,734562,2485,1999-11-01,'Action'
...,...,...,...,...,...
9187,527900,44,44,2016-09-14,"'Adventure', 'Casual'"
9188,528580,0,0,2000-02-03,Desconocido
9189,528660,182,182,2016-09-15,"'Action', 'Casual', 'Indie'"
9190,529820,0,0,2000-02-03,Desconocido


In [ ]:
dat_func1 = dat_func1.explode('genres')
dat_func1['year'] = pd.to_datetime(dat_func1['release_date']).dt.year
dat_func1_grouped = dat_func1.groupby(['year', 'genres']).agg({'playtime_forever': 'sum'}).reset_index()
genres_unique = dat_func1['genres'].unique()
result = pd.DataFrame({'genres': genres_unique}).merge(dat_func1_grouped, on='genres', how='left')
result = result.rename(columns={'year': 'year', 'playtime_forever': 'playtime_forever_sum'}).fillna(0)
idx_max_playtime = result.groupby('genres')['playtime_forever_sum'].idxmax()
dat_func1 = result.loc[idx_max_playtime, ['genres', 'year', 'playtime_forever_sum']]
dat_func1

,genres,year,playtime_forever_sum
22,'Action',2012.0,802222438
55,"'Action', 'Adventure'",2015.0,93692189
656,"'Action', 'Adventure', 'Casual'",2016.0,118817
1444,"'Action', 'Adventure', 'Casual', 'Free to Play...",2017.0,50953961
1685,"'Action', 'Adventure', 'Casual', 'Free to Play...",2015.0,37397
...,...,...,...
981,'Utilities',2014.0,2057576
1203,"'Utilities', 'Video Production'",2014.0,94754
1498,'Video Production',2015.0,354209
1130,'Web Publishing',2005.0,93847


##def UserForGenre( genero : str ):
**Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.**


crearemos dos dataframes reducidos con las columnas necesarias de *pd_items* y de *pd_reviews*

In [ ]:
pd_items_red = pd_items[['user_id', 'item_id', 'playtime_forever']]
pd_items_red

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5153204,76561198329548331,346330,0
5153205,76561198329548331,373330,0
5153206,76561198329548331,388490,3
5153207,76561198329548331,521570,4


In [ ]:
pd_reviews_red = pd_reviews[['user_id','item_id','posted']]
pd_reviews_red

,user_id,item_id,posted
0,76561198081848869,242860,2016-09-18
1,7656119815549OO43,227940,2016-09-18
2,reijimate,384490,2016-09-18
3,76561198066588605,381210,2016-09-18
4,76561198138586856,620,2016-09-18
...,...,...,...
58393,farsel,25700,2010-11-20
58394,farsel,15320,2010-11-20
58395,farsel,24010,2010-11-19
58396,eddy96,40700,2010-10-25


combinamos los subconjuntos

In [ ]:
merged_data = pd.merge(pd_reviews_red, pd_items_red, on=['user_id', 'item_id'], how='inner')
dat_func2=merged_data
dat_func2

,user_id,item_id,posted,playtime_forever
0,76561198081848869,242860,2016-09-18,462
1,7656119815549OO43,227940,2016-09-18,6
2,reijimate,384490,2016-09-18,322
3,76561198066588605,381210,2016-09-18,2242
4,76561198138586856,620,2016-09-18,504
...,...,...,...,...
44866,farsel,25700,2010-11-20,9
44867,farsel,15320,2010-11-20,36
44868,farsel,24010,2010-11-19,85
44869,eddy96,40700,2010-10-25,314


Traemos la columna genres de *pd_games* y la refinamos

In [ ]:

dat_func2_columns = ['user_id', 'item_id', 'posted', 'playtime_forever']
pd_games_columns = ['id', 'genres']


merged_df = pd.merge(dat_func2[dat_func2_columns], pd_games[pd_games_columns], how='left', left_on='item_id', right_on='id')
merged_df.drop(columns=['id'], inplace=True)
merged_df['genres'] = merged_df['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else 'desconocido' if pd.isnull(x) else x)
merged_df['genres'] = merged_df['genres'].str.strip("[]").astype(str)
dat_func2=merged_df
dat_func2

,user_id,item_id,posted,playtime_forever,genres
0,76561198081848869,242860,2016-09-18,462,"'Action', 'Indie', 'Strategy'"
1,7656119815549OO43,227940,2016-09-18,6,"'Action', 'Free to Play', 'Indie', 'Massively ..."
2,reijimate,384490,2016-09-18,322,"'Action', 'Adventure', 'Indie'"
3,76561198066588605,381210,2016-09-18,2242,desconocido
4,76561198138586856,620,2016-09-18,504,"'Action', 'Adventure'"
...,...,...,...,...,...
44866,farsel,25700,2010-11-20,9,'Indie'
44867,farsel,15320,2010-11-20,36,'Simulation'
44868,farsel,24010,2010-11-19,85,'Simulation'
44869,eddy96,40700,2010-10-25,314,"'Adventure', 'Indie'"


normalizamos los datos y desanidamos las listas de genres para obtener un valor unico

In [ ]:
dat_func2['genres'] = dat_func2['genres'].replace("Desconocido", "'desconocido'")
dat_func2['genres'] = dat_func2['genres'].replace("desconocido", "'desconocido'")
dat_func2['genres'] = dat_func2['genres'].apply(lambda x: eval(x))
dat_func2_expanded = dat_func2.explode('genres')
result = dat_func2_expanded.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()



Nos quedamos solo con el usuario que mas minutos (valor de playtime_forever) para cada genero

In [ ]:
idx_max_playtime = dat_func2_expanded.groupby('genres')['playtime_forever'].idxmax()
result = dat_func2_expanded.loc[idx_max_playtime, ['genres', 'user_id', 'playtime_forever']]

dat_func2 = result
dat_func2

,genres,user_id,playtime_forever
10123,Action,shinomegami,530882
10123,Free to Play,shinomegami,530882
10123,Massively Multiplayer,shinomegami,530882
10123,RPG,shinomegami,530882
10123,Strategy,shinomegami,530882
...,...,...,...
6873,Utilities,BewareWeLive,83352
16012,Video Production,Kipikinson,52768
11035,Design &amp; Illustration,Xyphien,71011
11035,Web Publishing,Xyphien,71011


los convertimos en csv para trabajarlo en las funciones

In [ ]:
dat_func1.to_csv('PlaytimeGenre.csv', index = False)
dat_func2.to_csv('UserForGenre.csv', index = False)

## def UsersRecommend( año : int ): ##
Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

## def UsersWorstDeveloper( año : int ):
Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

puesto que ambas piden datos correspondientes al sentimiento de analisis hecho previamente, armaremos un dataframe en conjunto para ambasy luego lo desglosaremos en dos Dataframes mucho mas pequeños que nos sirvan

Tomamos las columnas que nos interesan de *pd_games* y *pd_reviews*

In [ ]:
columnas_juegos = pd_games[['developer','id']]
columnas_reviews = pd_reviews[['user_id','item_id','posted','sentimiento']]

In [ ]:
dat_func3y4 = pd.merge(columnas_reviews, columnas_juegos, left_on='item_id', right_on='id', how='inner')
dat_func3y4.drop(columns= ['id', 'user_id'])

,item_id,posted,sentimiento,developer
0,242860,2016-09-18,1.0,"M2H,Blackmill Games"
1,242860,2016-09-15,2.0,"M2H,Blackmill Games"
2,242860,2016-09-05,0.0,"M2H,Blackmill Games"
3,242860,2016-09-02,2.0,"M2H,Blackmill Games"
4,242860,2016-08-20,2.0,"M2H,Blackmill Games"
...,...,...,...,...
53161,36000,2011-07-09,2.0,Sakari Indie
53162,58200,2011-07-04,2.0,Brawsome
53163,50510,2011-06-24,2.0,doublesix
53164,25900,2010-12-18,2.0,1C Company


a continuacion generaremos una tabla que nos devuelva, para cada año, las 3desarrolladoras con mejores y peores reviews usando el sentimiento como metrica


In [ ]:
dat_func3y4['posted'] = pd.to_datetime(dat_func3y4['posted'])
dat_func3y4['year'] = dat_func3y4['posted'].dt.year
sorted_data = dat_func3y4.sort_values(by=['year', 'sentimiento'], ascending=[True, False])
sorted_data


,user_id,item_id,posted,sentimiento,developer,id,year
5242,76561198025231200,440,2010-12-20,2.0,Valve,440,2010
5243,lemnisca,440,2010-12-20,2.0,Valve,440,2010
5244,elheat,440,2010-12-19,2.0,Valve,440,2010
5246,supernashwanpower,440,2010-12-18,2.0,Valve,440,2010
5247,Lone_walker,440,2010-12-18,2.0,Valve,440,2010
...,...,...,...,...,...,...,...
49037,serialies,266210,2016-01-03,0.0,Smoking WOLF,266210,2016
49043,76561197981203305,410210,2016-01-03,0.0,PiGravity,410210,2016
49067,76561198193227423,327130,2016-01-03,0.0,Digitalmindsoft,327130,2016
49068,76561198046868249,292120,2016-01-03,0.0,Square Enix,292120,2016


a partir del dataframe *grouped data* creamos el datafreme *func3*, este contiene las 3 mejores desarrolladoras por año y su puntaje respectivo

In [ ]:
grouped_data = sorted_data.groupby(['year', 'developer'])['sentimiento'].sum().reset_index()
grouped_data = grouped_data.sort_values(by=['year', 'sentimiento'], ascending=[True, False])
func3_list = []

for year in sorted_data['year'].unique():
    top_developers_year = grouped_data[grouped_data['year'] == year].head(3)
    top_developers_year.reset_index(drop=True, inplace=True)

    func3_year = pd.DataFrame({
        'year': [year],
        'best1': [top_developers_year.iloc[0]['developer'] if len(top_developers_year) > 0 else None],
        'score1': [top_developers_year.iloc[0]['sentimiento'] if len(top_developers_year) > 0 else None],
        'best2': [top_developers_year.iloc[1]['developer'] if len(top_developers_year) > 1 else None],
        'score2': [top_developers_year.iloc[1]['sentimiento'] if len(top_developers_year) > 1 else None],
        'best3': [top_developers_year.iloc[2]['developer'] if len(top_developers_year) > 2 else None],
        'score3': [top_developers_year.iloc[2]['sentimiento'] if len(top_developers_year) > 2 else None],
    })

    func3_list.append(func3_year)
func3 = pd.concat(func3_list, ignore_index=True)

In [ ]:
grouped_data = sorted_data.groupby(['year', 'developer'])['sentimiento'].sum().reset_index()
grouped_data = grouped_data.sort_values(by=['year', 'sentimiento'], ascending=[True, True])
func4_list = []

for year in sorted_data['year'].unique():
    bottom_developers_year = grouped_data[grouped_data['year'] == year].head(3)
    bottom_developers_year.reset_index(drop=True, inplace=True)

    func4_year = pd.DataFrame({
        'year': [year],
        'worst1': [bottom_developers_year.iloc[0]['developer'] if len(bottom_developers_year) > 0 else None],
        'score1': [bottom_developers_year.iloc[0]['sentimiento'] if len(bottom_developers_year) > 0 else None],
        'worst2': [bottom_developers_year.iloc[1]['developer'] if len(bottom_developers_year) > 1 else None],
        'score2': [bottom_developers_year.iloc[1]['sentimiento'] if len(bottom_developers_year) > 1 else None],
        'worst3': [bottom_developers_year.iloc[2]['developer'] if len(bottom_developers_year) > 2 else None],
        'score3': [bottom_developers_year.iloc[2]['sentimiento'] if len(bottom_developers_year) > 2 else None],
    })

    func4_list.append(func4_year)

func4 = pd.concat(func4_list, ignore_index=True)
func4

,year,worst1,score1,worst2,score2,worst3,score3
0,2010,DICE,0.0,Gas Powered Games,0.0,Playbrains,0.0
1,2011,11 bit studios,0.0,1C Company,0.0,Airtight Games,0.0
2,2012,800 North and Digital Ranch,0.0,Artery Games,0.0,Crankshaft Games,0.0
3,2013,14° East,0.0,2K Marin,0.0,3 Sprockets,0.0
4,2014,"2K Australia,Gearbox Software,Aspyr (Mac,Linux)",0.0,8-Bit Studio,0.0,Aleksey Abramenko,0.0
5,2015,2D BOY,0.0,"2K Australia,Gearbox Software,Aspyr (Mac,Linux)",0.0,3000AD,0.0
6,2016,16bit Nights,0.0,1C: Maddox Games,0.0,Andrew Gilmour,0.0


In [ ]:
func3.to_csv('UsersRecommend.csv', index = False)
func4.to_csv('UsersWorstDeveloper.csv', index = False)


## def sentiment_analysis( empresa desarrolladora : str ): ##

Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

tomaremos las columnas pertinetnes de los dataframes *pd_games* y *pd_reviews* para formar los datos que necesitara esta funcion

In [ ]:
pd_games5 = pd_games[['developer','id']]
pd_reviews5 = pd_reviews[['item_id','sentimiento']]
func5 = pd.merge(pd_games5, pd_reviews5, left_on ='id', right_on='item_id', how = 'inner')
func5

,developer,id,item_id,sentimiento
0,Stainless Games Ltd,282010,282010,1.0
1,Valve,70,70,2.0
2,Valve,70,70,2.0
3,Valve,70,70,2.0
4,Valve,70,70,1.0
...,...,...,...,...
53161,Valve,80,80,1.0
53162,Valve,80,80,1.0
53163,Valve,80,80,1.0
53164,Valve,80,80,2.0


In [ ]:
def categorizar_sentimiento(valor):
    if valor == 2.0:
        return 'positivo'
    elif valor == 1.0:
        return 'neutral'
    else:
        return 'negativo'

In [ ]:
func5['sentimiento_categoria'] = func5['sentimiento'].apply(categorizar_sentimiento)
func5 = func5.groupby(['developer', 'sentimiento_categoria']).size().unstack(fill_value=0)
func5.columns = ['negativo', 'neutral', 'positivo']
func5

,negativo,neutral,positivo
developer,,,
07th Expansion,1,2,0
"10th Art Studio,Adventure Productions",1,0,1
10tons Ltd,0,0,1
11 bit studios,28,9,25
14° East,1,0,1
...,...,...,...
xXarabongXx,0,1,0
△○□× (Miwashiba),0,0,5
"インレ,Inre",1,1,3


In [ ]:
func5.to_csv('sentiment_analysis.csv')